In [1]:
import sys
from pathlib import Path

#Path.cwd().parent depends on where you run this code from
#Path(__file__).resolve().parents[1] depends on where this file is located on disk
#
#project_root = .../1099-reconciliation-pipeline
project_root = Path.cwd().parent  # running from notebooks/  folder (cwd = current working directory) - .parent gets us to the folder above
sys.path.append(str(project_root)) #sys.path is a list of folders where python looks for modules
                                    # we add the project root to that list with .append

print(project_root)

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline


In [4]:
from src import load_data
from src.clean_matrix import clean_matrix
from src.clean_relius_roth_basis import clean_relius_roth_basis
from src.build_correction_file import build_correction_dataframe, write_correction_file

from src.config import RAW_DATA_DIR

relius_roth_basis_path = RAW_DATA_DIR / "real_roth_basis_relius_2025.xlsx"
matrix_path = RAW_DATA_DIR / "real_all_matrix_2025.xlsx"

# Load  and clean Matrix raw data
matrix_raw = load_data.load_matrix_excel(path=matrix_path, use_sample_if_none=False)
matrix_clean = clean_matrix(matrix_raw)

# Load  and clean Relius raw data
relius_roth_basis_raw = load_data.load_relius_roth_basis_excel(path=relius_roth_basis_path, use_sample_if_none=False, sheet_name="Sheet5")
relius_roth_basis_clean = clean_relius_roth_basis(relius_roth_basis_raw)


#.shape is an attribute of pandas DataFrames that returns a tuple of
# (number of rows, number of columns) - e.g. (1000, 15)
relius_roth_basis_clean.shape

/Users/manuelreyes/Desktop/dev/1099_reconciliation_pipeline/src/clean_matrix.py:177: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(series, errors="coerce").dt.date


(7904, 6)

In [5]:
relius_roth_basis_clean.head(10)

,plan_id,ssn,first_name,last_name,first_roth_tax_year,roth_basis_amt
0,100MBDII-R,201746826,Alyssa,Mihalik,2020,2500.0
5277,406MBDII-R,208726408,Josh,Myers,2021,9600.0
5275,406MBDII-R,167788536,Nina,Zanias-Anderson,2022,8400.0
5274,406MBDII-R,192721010,Juliana,Rappo,2018,7700.0
5273,406MBDII-R,165788891,Jacob,Augustus,2022,7700.0
5272,406MBDII-R,205769452,Zackary,Hockman,2022,7300.0
5271,406MBDII-R,530980202,Tristan,Boyd,2022,6975.0
5270,406MBDII-R,158784228,Elizabeth,Burks,2021,6650.0
5269,406MBDII-R,099763952,Katherine,Golebiewski,2023,6400.0
5268,406MBDII-R,197749337,Dana,Salanik,2023,6300.0


In [6]:
relius_roth_basis_clean["ssn"].str.len().value_counts(dropna=False)

ssn
9    7904
Name: count, dtype: int64

In [7]:
relius_roth_basis_clean.duplicated(["plan_id", "ssn"]).sum()

np.int64(0)

In [8]:
relius_roth_basis_clean.dtypes

plan_id                 object
ssn                     object
first_name              object
last_name               object
first_roth_tax_year      Int64
roth_basis_amt         float64
dtype: object